In [1]:
import heapq
import random
import numpy as np

### Settings

In [2]:
K = 50
N = 1000
d = 50
rho = 0.5
threshold = 0.001
sim = lambda x, y: np.sqrt(np.sum((x - y)**2))

### Dataset

In [3]:
V = np.array([np.random.randn(d) for _ in range(N)])
print(V.shape)

(1000, 50)


### NN-Descent

In [4]:
def reverse(B):
    Ro, Rn = [[] for _ in range(N)], [[] for _ in range(N)]
    for v in range(N):
        for dist, node, flg_new in B[v]:
            if flg_new:
                Rn[node].append(v)
            else:
                Ro[node].append(v)
    return Ro, Rn

def sample(S, n):
    random.shuffle(S)
    return S[:n]

In [5]:
# Graph Initialize
tmp = [i for i in range(N)]
B = [sorted([sim(V[x], V[y]), x, True] for x in random.sample(tmp, K)) for y in range(N)]

In [6]:
# Algorithm
rnd = 0
changes = N * K
while changes > threshold * N * K:
    rnd += 1
    changes = 0
    Bo, Bn = [], []
    Ro, Rn = reverse(B)
    for v in range(N):
        old, new = [], []
        for idx, (d, node, flg_new) in enumerate(B[v]):
            if flg_new:
                new.append((node, idx))
            else:
                old.append(node)
        newsample = []
        for node, i in sample(new, int(rho*K)):
            B[v][i][2] = False
            newsample.append(node)
        Bo.append(old)
        Bn.append(newsample)
        
    for v in range(N):
        Ao = set(Bo[v]).union(set(sample(Ro[v], int(rho*K))))
        An = set(Bn[v]).union(set(sample(Rn[v], int(rho*K))))
        for u1 in An:
            for u2 in An:
                if u1 >= u2:
                    continue
                distance = sim(V[u1], V[u2])
                heapq.heappush(B[u1], [distance, u2, True])
                heapq.heappush(B[u2], [distance, u1, True])
            for u2 in Ao:
                distance = sim(V[u1], V[u2])
                heapq.heappush(B[u1], [distance, u2, True])
                heapq.heappush(B[u2], [distance, u1, True])
        
    for v in range(N):
        tmp = []
        bef = -1
        cur = [-1, -1, -1]
        for _ in range(K):
            while cur[1] == bef or cur[1] == v:
                cur = heapq.heappop(B[v])
            tmp.append(cur)
            bef = cur[1]
        B[v] = tmp
        changes += sum(1 for d, n, flg in B[v] if flg)